In [2]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vidya\AppData\Roaming\nltk_data...


True

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


# Load the dataset
file_path = r"C:\Users\Vidya\Desktop\vidya\text.csv"
df = pd.read_csv(file_path)


In [3]:
df.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [6]:
df.shape

(416809, 3)

In [10]:
df['label'].value_counts()

label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64

In [7]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Retain certain punctuation marks that may convey emotions
    retain_punctuation = ['!', '?', '.', ',', ';', ':', '(', ')', "'", '"']
    text = ''.join([char if char.isalnum() or char in retain_punctuation else ' ' for char in text])

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]

    # Join stemmed tokens back into a string
    preprocessed_text = ' '.join(stemmed_tokens)

    return preprocessed_text

# Apply preprocessing to the text column
df['text'] = df['text'].apply(preprocess_text)

# Tokenize and pad sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
X = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(X, maxlen=100)

# Encode labels
encoder = LabelEncoder()
y = encoder.fit_transform(df['label'])



In [8]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=16, input_shape=(100, )))
model.add(SimpleRNN(units=64))
model.add(Dense(units=6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 16)        │       829,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 64)             │         5,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 835,510 (3.19 MB)

 Trainable params: 835,510 (3.19 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)



Epoch 1/10
9379/9379 ━━━━━━━━━━━━━━━━━━━━ 142s 15ms/step - accuracy: 0.7531 - loss: 0.6615 - val_accuracy: 0.8966 - val_loss: 0.2671
Epoch 2/10
9379/9379 ━━━━━━━━━━━━━━━━━━━━ 147s 16ms/step - accuracy: 0.9090 - loss: 0.2223 - val_accuracy: 0.9054 - val_loss: 0.2092
Epoch 3/10
9379/9379 ━━━━━━━━━━━━━━━━━━━━ 202s 16ms/step - accuracy: 0.9158 - loss: 0.1886 - val_accuracy: 0.9079 - val_loss: 0.1908
Epoch 4/10
9379/9379 ━━━━━━━━━━━━━━━━━━━━ 203s 16ms/step - accuracy: 0.9159 - loss: 0.1892 - val_accuracy: 0.9055 - val_loss: 0.2014
Epoch 5/10
9379/9379 ━━━━━━━━━━━━━━━━━━━━ 141s 15ms/step - accuracy: 0.9190 - loss: 0.1750 - val_accuracy: 0.9072 - val_loss: 0.1961
Epoch 6/10
9379/9379 ━━━━━━━━━━━━━━━━━━━━ 141s 15ms/step - accuracy: 0.9207 - loss: 0.1764 - val_accuracy: 0.9100 - val_loss: 0.1890
Epoch 7/10
9379/9379 ━━━━━━━━━━━━━━━━━━━━ 143s 15ms/step - accuracy: 0.9241 - loss: 0.1606 - val_accuracy: 0.9067 - val_loss: 0.2222
Epoch 8/10
9379/9379 ━━━━━━━━━━━━━━━━━━━━ 142s 15ms/step - accuracy: 

In [10]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

2606/2606 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9093 - loss: 0.1918
Test Loss: 0.1945856660604477
Test Accuracy: 0.9084115028381348


Random Forest multi-class classifier on the RNN representations

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder



In [23]:
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
 

In [24]:
random_forest.fit(X_train, y_train)



RandomForestClassifier(random_state=42)

In [26]:
y_pred = random_forest.predict(X_test)


In [27]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))

Test Accuracy: 0.41612485305055064
              precision    recall  f1-score   support

           0       0.45      0.50      0.47     24201
           1       0.41      0.70      0.52     28164
           2       0.19      0.06      0.09      6929
           3       0.46      0.13      0.20     11441
           4       0.33      0.10      0.15      9594
           5       0.14      0.05      0.07      3033

    accuracy                           0.42     83362
   macro avg       0.33      0.26      0.25     83362
weighted avg       0.39      0.42      0.37     83362

